In [ ]:
### Import libraries
from netCDF4 import Dataset
import xarray as xr
import numpy as np
import cartopy.crs as crs
import matplotlib
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
from wrf import getvar, interplevel, smooth2d
import warnings
from scipy.interpolate import griddata
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
warnings.filterwarnings('ignore')
import matplotlib.ticker as mticker
import matplotlib.transforms as mtransforms

CP = 1005.7
RD = 287.04
P0 = 1000.
TR = 300.
LV = 2.501e6
EPS = 1.


In [ ]:
### Read in the data
lats = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/lats.npy')
lons = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/lons.npy')

zl1_ctl = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/ctl/z500_11-14_00.npy')
zl2_ctl = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/ctl/z500_11-14_12.npy')
zl3_ctl = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/ctl/z500_11-15_00.npy')
zl4_ctl = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/ctl/z500_11-15_12.npy')

dbz1_ctl = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/ctl/dbz_11-14_00.npy')
dbz2_ctl = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/ctl/dbz_11-14_12.npy')
dbz3_ctl = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/ctl/dbz_11-15_00.npy')
dbz4_ctl = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/ctl/dbz_11-15_12.npy')

vl1_ctl = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/ctl/v500_11-14_00.npy')
vl2_ctl = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/ctl/v500_11-14_12.npy')
vl3_ctl = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/ctl/v500_11-15_00.npy')
vl4_ctl = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/ctl/v500_11-15_12.npy')

vl1_wave = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/wave/v500_11-14_00.npy')
vl2_wave = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/wave/v500_11-14_12.npy')
vl3_wave = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/wave/v500_11-15_00.npy')
vl4_wave = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/wave/v500_11-15_12.npy')

dv1 = vl1_wave - vl1_ctl
dv2 = vl2_wave - vl2_ctl
dv3 = vl3_wave - vl3_ctl
dv4 = vl4_wave - vl4_ctl


In [ ]:
### Plot subdomain
min_lat = 15
max_lat = 45
min_lon = -105
max_lon = -65

### Contour interval
vlvls = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])

### Cartopy projection
proj = crs.PlateCarree()

fig, axd = plt.subplot_mosaic([['topleft','topright','cbar'],['bottomleft','bottomright','cbar']],
                              constrained_layout=True, figsize=(7.2,5.2), dpi=1000, 
                              gridspec_kw={'width_ratios':[1,1,0.05],'height_ratios':[1,1]},
                              per_subplot_kw={'topleft':{'projection':proj},
                                              'topright':{'projection':proj},
                                              'bottomleft':{'projection':proj},
                                              'bottomright':{'projection':proj}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### TOP LEFT
#############

# Add the gridlines
gls = axd['topleft'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topleft'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-100, -90, -80, -70])
gls.ylocator = mticker.FixedLocator([20, 30, 40])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=False
gls.left_labels=True

cs1 = axd['topleft'].contour(lons, lats, smooth2d(zl1_ctl,2)/10.,
                  levels=np.arange(0,1500,15),alpha=0.9,
                  colors='k', transform=crs.PlateCarree())
axd['topleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im1 = axd['topleft'].contourf(lons, lats, dv1, levels=vlvls,
            cmap = get_cmap('RdBu_r'), extend='both',zorder=-20,
            transform=crs.PlateCarree())
axd['topleft'].set_rasterization_zorder(-18)
im0 = axd['topleft'].contourf(lons, lats, dbz1_ctl, levels=[5,5.01],
            colors='k', extend='max',alpha=0.2,zorder=-19,
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topleft'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topleft'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topleft'].set_title('0000 UTC 14 Nov 2018 ($t$ = 12 h)')
axd['topleft'].text(0.0, 1.0, '(a)',transform=axd['topleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### TOP RIGHT
#############

gls = axd['topright'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topright'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-100, -90, -80, -70])
gls.ylocator = mticker.FixedLocator([20, 30, 40])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=False
gls.left_labels=False

cs1 = axd['topright'].contour(lons, lats, smooth2d(zl2_ctl,2)/10.,
                  levels=np.arange(0,1500,15),alpha=0.9,
                  colors='k', transform=crs.PlateCarree())
axd['topright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im1 = axd['topright'].contourf(lons, lats, dv2, levels=vlvls,
            cmap = get_cmap('RdBu_r'), extend='both',zorder=-20,
            transform=crs.PlateCarree())
axd['topright'].set_rasterization_zorder(-18)
im0 = axd['topright'].contourf(lons, lats, dbz2_ctl, levels=[5,5.01],
            colors='k', extend='max',alpha=0.2,zorder=-19,
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topright'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topright'].set_title('1200 UTC 14 Nov 2018 ($t$ = 24 h)')
axd['topright'].text(0.0, 1.0, '(b)',transform=axd['topright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM LEFT
#############

gls = axd['bottomleft'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottomleft'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-100, -90, -80, -70])
gls.ylocator = mticker.FixedLocator([20, 30, 40])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.left_labels=True

cs1 = axd['bottomleft'].contour(lons, lats, smooth2d(zl3_ctl,2)/10.,
                  levels=np.arange(0,1500,15),alpha=0.9,
                  colors='k', transform=crs.PlateCarree())
axd['bottomleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im1 = axd['bottomleft'].contourf(lons, lats, dv3, levels=vlvls,
            cmap = get_cmap('RdBu_r'), extend='both',zorder=-20,
            transform=crs.PlateCarree())
axd['bottomleft'].set_rasterization_zorder(-18)
im0 = axd['bottomleft'].contourf(lons, lats, dbz3_ctl, levels=[5,5.01],
            colors='k', extend='max',alpha=0.2,zorder=-19,
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomleft'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottomleft'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottomleft'].set_title('0000 UTC 15 Nov 2018 ($t$ = 36 h)')
axd['bottomleft'].text(0.0, 1.0, '(c)',transform=axd['bottomleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM RIGHT
#############

gls = axd['bottomright'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottomright'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-100, -90, -80, -70])
gls.ylocator = mticker.FixedLocator([20, 30, 40])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.left_labels=False

cs1 = axd['bottomright'].contour(lons, lats, smooth2d(zl4_ctl,2)/10.,
                  levels=np.arange(0,1500,15),alpha=0.9,
                  colors='k', transform=crs.PlateCarree())
axd['bottomright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im1 = axd['bottomright'].contourf(lons, lats, dv4, levels=vlvls,
            cmap = get_cmap('RdBu_r'), extend='both',zorder=-20,
            transform=crs.PlateCarree())
axd['bottomright'].set_rasterization_zorder(-18)
im0 = axd['bottomright'].contourf(lons, lats, dbz4_ctl, levels=[5,5.01],
            colors='k', extend='max',alpha=0.2,zorder=-19,
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomright'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottomright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottomright'].set_title('1200 UTC 15 Nov 2018 ($t$ = 48 h)')
axd['bottomright'].text(0.0, 1.0, '(d)',transform=axd['bottomright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

### SET THE COLORBAR AND SHOW
cbar = fig.colorbar(im1, orientation='vertical', cax=axd['cbar'])
plt.savefig('/p/work1/lloveras/real/nov2018/figs_nov2018/4km_wave.pdf',bbox_inches='tight')
plt.show()
